## Importing Libraries 

In [2]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import datetime as dt
import numpy as np
import hvplot.pandas
import panel as pn
import plotly.express as px
pn.extension("plotly")
%matplotlib inline

## Importing Data from CSV Files

In [3]:
xlk_csv = pd.read_csv("../Project_1/xlk.csv", index_col = "Date", infer_datetime_format = True, 
                      parse_dates = True)
xlf_csv = pd.read_csv("../Project_1/XLF.csv", index_col = "Date", infer_datetime_format = True, 
                      parse_dates = True)
vnq_csv = pd.read_csv("../Project_1/VNQ.csv", index_col = "Date", infer_datetime_format = True, 
                      parse_dates = True)
spy_csv = pd.read_csv("../Project_1/spy.csv", index_col = "Date", infer_datetime_format = True, 
                      parse_dates = True)
ibb_csv = pd.read_csv("../Project_1/ibb.csv", index_col = "Date", infer_datetime_format = True, 
                      parse_dates = True)
covid_data = pd.read_csv("United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv", 
                         index_col="submission_date", parse_dates=True)

## Creating Dataframe for ETF Prices Pre-Covid¶

In [11]:
xlk_pre_covid = xlk_csv.reset_index()
xlk_pre_covid = xlk_pre_covid[0:767]
xlk_pre_covid.columns = ['Date', 'Tech']

xlf_pre_covid = xlf_csv.reset_index()
xlf_pre_covid = xlf_pre_covid[0:767]
xlf_pre_covid.columns = ['Date', 'Finance']
xlf_pre_covid = xlf_pre_covid.drop(columns=['Date'])

vnq_pre_covid = vnq_csv.reset_index()
vnq_pre_covid = vnq_pre_covid[0:767]
vnq_pre_covid.columns = ['Date', 'Real_Estate']
vnq_pre_covid = vnq_pre_covid.drop(columns=['Date'])

spy_pre_covid = spy_csv.reset_index()
spy_pre_covid = spy_pre_covid[0:767]
spy_pre_covid.columns = ['Date', 'SNP500']
spy_pre_covid = spy_pre_covid.drop(columns=['Date'])

ibb_pre_covid = ibb_csv.reset_index()
ibb_pre_covid = ibb_pre_covid[0:767]
ibb_pre_covid.columns = ['Date', 'BioTech']
ibb_pre_covid = ibb_pre_covid.drop(columns=['Date'])

pre_covid_concat = pd.concat([xlk_pre_covid, xlf_pre_covid, vnq_pre_covid,
                              spy_pre_covid, ibb_pre_covid], axis = "columns", join = "inner")
pre_covid_concat.set_index(["Date"], inplace = True)

## Creating Dataframe for ETF Prices During Covid

In [12]:
xlk_post_covid = xlk_csv.reset_index()
xlk_post_covid = xlk_post_covid[767:]
xlk_post_covid.columns = ['Date', 'Tech']

xlf_post_covid = xlf_csv.reset_index()
xlf_post_covid = xlf_post_covid[767:]
xlf_post_covid.columns = ['Date', 'Finance']
xlf_post_covid = xlf_post_covid.drop(columns=['Date'])

vnq_post_covid = vnq_csv.reset_index()
vnq_post_covid = vnq_post_covid[767:]
vnq_post_covid.columns = ['Date', 'Real_Estate']
vnq_post_covid = vnq_post_covid.drop(columns=['Date'])

spy_post_covid = spy_csv.reset_index()
spy_post_covid = spy_post_covid[767:]
spy_post_covid.columns = ['Date', 'SNP500']
spy_post_covid = spy_post_covid.drop(columns=['Date'])

ibb_post_covid = ibb_csv.reset_index()
ibb_post_covid = ibb_post_covid[767:]
ibb_post_covid.columns = ['Date', 'BioTech']
ibb_post_covid = ibb_post_covid.drop(columns=['Date'])

during_covid_concat = pd.concat([xlk_post_covid, xlf_post_covid, vnq_post_covid,
                              spy_post_covid, ibb_post_covid], axis = "columns", join = "inner")
during_covid_concat.set_index(["Date"], inplace = True)

## Creating Covid Cases Dataframe

In [15]:
covid_clean_data = covid_data[["tot_cases","tot_death"]]
covid_really_clean = covid_clean_data.groupby(covid_clean_data.index).sum()
covid_really_clean.reset_index(inplace = True)
covid_really_clean.columns = ['Date', 'Total_Cases', 'Total_Deaths']
covid_really_clean.set_index('Date', inplace=True)

## Concatinating Covid Dataframe with ETF Prices

In [17]:
covid_data_impact = pd.concat([covid_really_clean, during_covid_concat], axis = 'columns',
                              join = 'inner'
                             )

## Creating the Dashboard

### Functions

In [33]:
def pre_covid_trend():
    """Trends of the data from 2017 to 2020 before covid hit"""
    graph = px.line(pre_covid_concat)
    return graph

def during_covid_trend():
    """Trends of the data in 2020 once Covid hit on January 21"""
    graph = px.line(during_covid_concat)
    return graph

def all_time_trend():
    """Trend of the data of the entire DataFrame"""
    pre_covid_plot = pre_covid_concat.hvplot()
    during_covid = during_covid_concat.hvplot()
    graph = pre_covid_plot * during_covid
    return graph

def sharpe_pre_covid():
    """Sharpe ratios for the ETF's pre covid"""
    daily_returns_precovid = pre_covid_concat.pct_change().dropna()
    std_pre_covid = pre_covid_concat.std()
    volatility = std_pre_covid*np.sqrt(252)
    correlation_pre_covid = pre_covid_concat.corr()
    correlation_pre_covid_sum = correlation_pre_covid.sum(axis=0).sort_values()
    sharpe_ratios_precovid = (daily_returns_precovid.mean()*252) / (daily_returns_precovid.std() * np.sqrt(252))
    sharpe_ratios_precovid.sort_values(ascending = False, inplace = True)
    graph = sharpe_ratios_precovid.hvplot.bar(title = 'Sharpe Ratios Pre Covid')
    return graph

def sharpe_during_covid():
    """Sharpe ratios for the ETF's during covid"""
    daily_returns_covid = during_covid_concat.pct_change().dropna()
    std_during_covid = during_covid_concat.std()
    during_volatility = std_during_covid*np.sqrt(252)
    correlation_covid = during_covid_concat.corr()
    correlation_covid_sum = correlation_covid.sum(axis=0).sort_values()
    sharpe_ratios_covid = (daily_returns_covid.mean()*252) / (daily_returns_covid.std() * np.sqrt(252))
    sharpe_ratios_covid.sort_values(ascending = False, inplace = True)
    graph = sharpe_ratios_covid.hvplot.bar(title = 'Sharpe Ratios During Covid')
    return graph

def sharpe_concatinated():
    """Combined Sharpe ratios for the ETF's before and during covid"""
    daily_returns_precovid = pre_covid_concat.pct_change().dropna()
    std_pre_covid = pre_covid_concat.std()
    volatility = std_pre_covid*np.sqrt(252)
    correlation_pre_covid = pre_covid_concat.corr()
    correlation_pre_covid_sum = correlation_pre_covid.sum(axis=0).sort_values()
    sharpe_ratios_precovid = (daily_returns_precovid.mean()*252) / (daily_returns_precovid.std() * np.sqrt(252))
    sharpe_ratios_precovid.sort_values(ascending = False, inplace = True)
    sharpe_pre_covid_graph = sharpe_ratios_precovid.hvplot.bar(title = 'Sharpe Ratios Pre Covid')
    daily_returns_covid = during_covid_concat.pct_change().dropna()
    std_during_covid = during_covid_concat.std()
    during_volatility = std_during_covid*np.sqrt(252)
    correlation_covid = during_covid_concat.corr()
    correlation_covid_sum = correlation_covid.sum(axis=0).sort_values()
    sharpe_ratios_covid = (daily_returns_covid.mean()*252) / (daily_returns_covid.std() * np.sqrt(252))
    sharpe_ratios_covid.sort_values(ascending = False, inplace = True)
    sharpe_covid_graph = sharpe_ratios_covid.hvplot.bar(title = 'Sharpe Ratios During Covid')
    graph = sharpe_pre_covid_graph * sharpe_covid_graph
    return graph

def beta_pre_covid():
    """Beta Values for the ETF's pre covid"""
    pre_covid_xlk_cov = pre_covid_concat['Tech'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_xlf_cov = pre_covid_concat['Finance'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_vnq_cov = pre_covid_concat['Real_Estate'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_ibb_cov = pre_covid_concat['BioTech'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_var = pre_covid_concat['SNP500'].rolling(window = 2).var().dropna()
    xlk_beta_pre_covid = pre_covid_xlk_cov/pre_covid_var
    xlf_beta_pre_covid = pre_covid_xlf_cov/pre_covid_var
    vnq_beta_pre_covid = pre_covid_vnq_cov/pre_covid_var
    ibb_beta_pre_covid = pre_covid_ibb_cov/pre_covid_var
    beta_pre_covid = pd.concat([xlk_beta_pre_covid, xlf_beta_pre_covid, vnq_beta_pre_covid, ibb_beta_pre_covid],
                          axis = 'columns', join = 'inner')
    graph = beta_pre_covid.hvplot(figsize = (10, 5), title = 'Beta Values Pre-Covid'
                                       ).opts(ylim=(-150, 220))
    return graph

def beta_during_covid():
    """Beta Values for the ETF's during covid"""
    during_covid_xlk_cov = during_covid_concat['Tech'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_xlf_cov = during_covid_concat['Finance'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_vnq_cov = during_covid_concat['Real_Estate'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_ibb_cov = during_covid_concat['BioTech'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_var = during_covid_concat['SNP500'].rolling(window = 2).var().dropna()
    xlk_beta_covid = during_covid_xlk_cov/during_covid_var
    xlf_beta_covid = during_covid_xlf_cov/during_covid_var
    vnq_beta_covid = during_covid_vnq_cov/during_covid_var
    ibb_beta_covid = during_covid_ibb_cov/during_covid_var
    pct_change_covid = covid_really_clean.pct_change()
    beta_during_covid = pd.concat([xlk_beta_covid, xlf_beta_covid, vnq_beta_covid, ibb_beta_covid],
                             axis = 'columns', join = 'inner')
    graph = beta_during_covid.hvplot(figsize = (20, 10), title = 'Beta During Covid')
    return graph

def beta_concatinated():
    """Beta Values for the ETF's before and during covid"""
    pre_covid_xlk_cov = pre_covid_concat['Tech'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_xlf_cov = pre_covid_concat['Finance'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_vnq_cov = pre_covid_concat['Real_Estate'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_ibb_cov = pre_covid_concat['BioTech'].rolling(window = 2).cov(pre_covid_concat['SNP500']).dropna()
    pre_covid_var = pre_covid_concat['SNP500'].rolling(window = 2).var().dropna()
    xlk_beta_pre_covid = pre_covid_xlk_cov/pre_covid_var
    xlf_beta_pre_covid = pre_covid_xlf_cov/pre_covid_var
    vnq_beta_pre_covid = pre_covid_vnq_cov/pre_covid_var
    ibb_beta_pre_covid = pre_covid_ibb_cov/pre_covid_var
    beta_pre_covid = pd.concat([xlk_beta_pre_covid, xlf_beta_pre_covid, vnq_beta_pre_covid, ibb_beta_pre_covid],
                          axis = 'columns', join = 'inner')
    beta_precovid_graph = beta_pre_covid.hvplot(figsize = (10, 5), title = 'Beta Values Pre-Covid'
                                       ).opts(ylim=(-150, 220))
    during_covid_xlk_cov = during_covid_concat['Tech'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_xlf_cov = during_covid_concat['Finance'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_vnq_cov = during_covid_concat['Real_Estate'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_ibb_cov = during_covid_concat['BioTech'].rolling(window = 2).cov(during_covid_concat['SNP500']).dropna()
    during_covid_var = during_covid_concat['SNP500'].rolling(window = 2).var().dropna()
    xlk_beta_covid = during_covid_xlk_cov/during_covid_var
    xlf_beta_covid = during_covid_xlf_cov/during_covid_var
    vnq_beta_covid = during_covid_vnq_cov/during_covid_var
    ibb_beta_covid = during_covid_ibb_cov/during_covid_var
    pct_change_covid = covid_really_clean.pct_change()
    beta_during_covid = pd.concat([xlk_beta_covid, xlf_beta_covid, vnq_beta_covid, ibb_beta_covid],
                             axis = 'columns', join = 'inner')
    beta_during_covid_graph = beta_during_covid.hvplot(figsize = (20, 10), title = 'Beta During Covid')
    graph = beta_precovid_graph + beta_during_covid_graph
    return graph


### Creating the Dashboard itself

In [ ]:
"""This section includes only pre covid graphs"""
pre_covid_info = pn.Column("All graphs pre covid", pre_covid_trend(), sharpe_pre_covid(),
                          beta_pre_covid())

"""This section includes graphs of only the covid period"""
covid_info = pn.Column("All graphs during covid", during_covid_trend(), sharpe_during_covid(),
                      beta_during_covid())

"""This section includes the concatinated graphs of both periods"""
all_info = pn.Column("The big picutre", all_time_trend(), sharpe_concatinated(),
                    beta_concatinated())

"""Dashboard visualisation"""
dashboard_covid = pn.Tabs(
    ("Pre-Covid", pre_covid_info), ("During Covid", covid_info), ("Comparisons", all_info)
)
dashboard_covid